In [ ]:
%matplotlib inline
import pandas as pd
import os
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
filepath = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'
art_nodes = pd.read_csv(os.path.join(filepath, 'article_nodes_nodup_min5_bimodal_titles.csv'))
case_nodes = pd.read_csv(os.path.join(filepath, 'case_nodes_simple_bimodal_titles.csv'))
bimodal_links = pd.read_csv(os.path.join(filepath, 'case_to_article_title_links.csv'))
bimodal_clusters = pd.read_csv(os.path.join(filepath, 'bimodal_clusters_titles.csv'))

In [ ]:
comm_counts = bimodal_clusters.groupby(['community', 'type']).count()['name'].unstack()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8, 15))

n = 50

comm_counts[:n].plot(kind='bar', ax=axes[0], title='Cases and articles')
comm_counts['article'][:n].plot(kind='bar', ax=axes[1], title='Articles')
comm_counts['case'][:n].plot(kind='bar', ax=axes[2], title='Cases')

In [ ]:
largest_coms = bimodal_clusters.community.value_counts().index

In [ ]:
df_plot = case_nodes[case_nodes.community_bimodal.isin(largest_coms[:10])]
df_plot.groupby(['year','community_bimodal']).count()['ecli'].unstack().plot(colormap='nipy_spectral')
plt.gca().set_xlim(1990, 2017)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

## compare with courts

In [ ]:
courts = pd.read_csv(os.path.join(filepath, 'court_nodes.csv'))

# Some mappings
courts.loc[courts['Type']=='TypeHr','Type'] = 'Hoge Raad'
courts.loc[courts['Type']=='TypeCBb','Type'] = 'College van beroep'
courts.loc[courts['Type']=='TypeCRvB','Type'] = 'Centrale Raad van Beroep'
courts.loc[courts['Type']=='TypeRvS','Type'] = 'Raad van State'
courts.loc[courts['Type']=='http://psi.rechtspraak.nl/buitenlandseInstantie','Type'] = 'Buitenlandse Instantie'

In [ ]:
courts.Type.value_counts()

In [ ]:
case_nodes_courts = pd.merge(case_nodes, courts, how='left', left_on='court', right_on='id')

In [ ]:
df_plot = case_nodes_courts[case_nodes_courts.community_bimodal.isin(largest_coms[:20])]
df_plot.groupby(['community_bimodal', 'Type']).count()['ecli'].unstack().plot.bar(stacked=True, colormap='nipy_spectral', figsize=(13,5))
plt.legend(bbox_to_anchor=(0, 1.1),loc=8, borderaxespad=0.)
plt.show()

In [ ]:
for com in largest_coms[:20]:
    nodes_sub = art_nodes[art_nodes.community==com]
    nodes_sub.groupby(['book']).count()['id'].plot.barh(figsize=(3,3))
    plt.title('Community {}, {} articles'.format(com, len(nodes_sub)))
    plt.show()
    #display(nodes_sub)